<a href="https://colab.research.google.com/github/riturai1997/DQN_Feature_Fusion/blob/main/final_dqn_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
import pandas as pd
import numpy as np
from scipy.stats import pearsonr, entropy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, accuracy_score, f1_score, roc_auc_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from collections import deque, defaultdict
import random
import time
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
def load_and_preprocess_dataset(zip_file_path, attack_files):
    df_list = []

    with zipfile.ZipFile(zip_file_path, 'r') as z:
        for file_name, label in attack_files.items():
            with z.open(file_name) as attack_file:
                attack_df = pd.read_csv(attack_file)

                # Add label column
                attack_df['Label'] = attack_df['Class'].apply(lambda x: 1 if x == 'T' else 0)


                # Oversample minority class
                t_class_size = attack_df[attack_df['Class'] == 'T'].shape[0]
                r_class_size = attack_df[attack_df['Class'] == 'R'].shape[0]
                majority_class = 'T' if t_class_size > r_class_size else 'R'
                minority_class = 'R' if t_class_size > r_class_size else 'T'

                if majority_class == 'T':
                    minority_samples = attack_df[attack_df['Class'] == minority_class].sample(n=t_class_size, replace=True, random_state=42)
                    majority_samples = attack_df[attack_df['Class'] == majority_class]
                else:
                    minority_samples = attack_df[attack_df['Class'] == minority_class].sample(n=r_class_size, replace=True, random_state=42)
                    majority_samples = attack_df[attack_df['Class'] == majority_class]
                minority_samples = minority_samples.sample(n=50000, random_state=42)
                majority_samples = majority_samples.sample(n=50000, random_state=42)
                attack_sampled_df = pd.concat([minority_samples, majority_samples], axis=0)
                df_list.append(attack_sampled_df)


    combined_df = pd.concat(df_list, axis=0)


    def remove_single_quote(value):
        return str(value).replace("'", "")

    def convert_exponential_to_hex(value):
        try:
            if isinstance(value, str) and 'E+' in value:
                value = float(value)
            hex_value = hex(int(value))[2:]
            return hex_value.zfill(4).upper()
        except ValueError:
            return value

    hex_columns = ['CAN ID','D[0]','D[1]','D[2]','D[3]','D[4]','D[5]','D[6]','D[7]']
    for col in hex_columns:
        combined_df[col] = combined_df[col].apply(remove_single_quote)
        combined_df[col] = combined_df[col].apply(convert_exponential_to_hex)
        combined_df[col] = combined_df[col].apply(lambda x: int(x, 16) if pd.notnull(x) else 0)

    return combined_df

In [ ]:

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


def add_advanced_features(df):
    """Add sophisticated features for anomaly detection"""
    # 1. Payload statistics
    payload_cols = ['D[0]','D[1]','D[2]','D[3]','D[4]','D[5]','D[6]','D[7]']
    df['Payload_Mean'] = df[payload_cols].mean(axis=1)
    df['Payload_Std'] = df[payload_cols].std(axis=1)
    df['Payload_Min'] = df[payload_cols].min(axis=1)
    df['Payload_Max'] = df[payload_cols].max(axis=1)

    # 2. Byte-level features
    for i in range(8):
        df[f'Byte_{i}_Delta'] = df[f'D[{i}]'].diff().abs().fillna(0)
        df[f'Byte_{i}_Rolling_Mean'] = df[f'D[{i}]'].rolling(5, min_periods=1).mean()

    # 3. Entropy and uniqueness
    df['Payload_Entropy'] = df[payload_cols].apply(lambda x: entropy(pd.Series(x).value_counts(normalize=True)), axis=1)
    df['Unique_Bytes'] = df[payload_cols].apply(lambda x: len(set(x)), axis=1)

    # 4. Message frequency features
    if 'Timestamp' in df.columns:
        df['Msg_Interval'] = df.groupby('CAN ID')['Timestamp'].diff().fillna(0)
        df['Msg_Frequency'] = 1/(df.groupby('CAN ID')['Msg_Interval'].transform('mean') + 1e-6)

    # 5. Advanced position-specific features
    df['Byte_0_Normalized'] = df['D[0]']/255
    df['Byte_1_2_Ratio'] = np.where(df['D[1]']!=0, df['D[2]']/df['D[1]'], 0)
    df['Byte_3_4_Diff'] = df['D[3]'] - df['D[4]']

    # 6. CAN ID specific features
    df['CAN_ID_Frequency'] = df.groupby('CAN ID')['CAN ID'].transform('count')
    df['CAN_ID_Entropy'] = df.groupby('CAN ID')['D[0]'].transform(lambda x: entropy(pd.Series(x).value_counts(normalize=True)))

    # 7. Payload change rate
    df['Payload_Change_Rate'] = df[payload_cols].diff(axis=1).abs().sum(axis=1)

    return df


Using device: cpu


In [ ]:

class EnhancedDQN(nn.Module):
    def __init__(self, state_size, action_size):
        super(EnhancedDQN, self).__init__()
        self.fc1 = nn.Linear(state_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, action_size)

        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)
        x = torch.relu(self.fc3(x))
        return self.fc4(x)

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=100000)
        self.gamma = 0.99
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.batch_size = 128

        self.model = EnhancedDQN(state_size, action_size).to(device)
        self.target_model = EnhancedDQN(state_size, action_size).to(device)
        self.target_model.load_state_dict(self.model.state_dict())
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)
        self.loss_fn = nn.SmoothL1Loss()

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if random.random() <= self.epsilon:
            return random.randint(0, self.action_size - 1)
        state = torch.FloatTensor(state).unsqueeze(0).to(device)
        self.model.eval()  # Set model to evaluation mode
        with torch.no_grad():
            act_values = self.model(state)
        return torch.argmax(act_values).item()

    def replay(self):
        if len(self.memory) < self.batch_size:
            return

        minibatch = random.sample(self.memory, self.batch_size)
        states = torch.FloatTensor(np.array([t[0] for t in minibatch])).to(device)
        actions = torch.LongTensor(np.array([t[1] for t in minibatch])).to(device)
        rewards = torch.FloatTensor(np.array([t[2] for t in minibatch])).to(device)
        next_states = torch.FloatTensor(np.array([t[3] for t in minibatch])).to(device)
        dones = torch.FloatTensor(np.array([t[4] for t in minibatch])).to(device)

        self.model.train()  # Set model to training mode
        current_q = self.model(states).gather(1, actions.unsqueeze(1))
        next_q = self.target_model(next_states).max(1)[0].detach()
        target = rewards + (1 - dones) * self.gamma * next_q

        loss = self.loss_fn(current_q.squeeze(), target)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def decay_epsilon(self):
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

    def update_target_model(self):
        self.target_model.load_state_dict(self.model.state_dict())

class CarHackingEnv:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.current_index = 0
        self.max_steps = len(X)

    def reset(self):
        self.current_index = 0
        return self.X[self.current_index]

    def step(self, action):
        true_label = self.y[self.current_index]
        reward = 20 if action == true_label else -10
        self.current_index += 1
        done = self.current_index >= self.max_steps
        next_state = self.X[self.current_index] if not done else None
        return next_state, reward, done, {'true_label': true_label}


In [ ]:
car_hacking_path = '/content/drive/My Drive/Survival_Analysis/SONATA.zip'
car_hacking_files = {
    'Flooding.csv': 2,
    'Malfunction.csv': 1,
    'Fuzzy.csv': 3
}

df = load_and_preprocess_dataset(car_hacking_path, car_hacking_files)
df = add_advanced_features(df)

In [ ]:
# Select features - now including all CAN ID and payload bytes plus engineered features
selected_features = [
    'CAN ID', 'D[0]', 'D[1]', 'D[2]', 'D[3]', 'D[4]', 'D[5]', 'D[6]', 'D[7]',
    'Payload_Mean', 'Payload_Std', 'Payload_Entropy', 'Unique_Bytes',
    'Msg_Frequency', 'Byte_1_2_Ratio', 'Byte_3_4_Diff', 'CAN_ID_Frequency',
    'CAN_ID_Entropy', 'Payload_Change_Rate'
]

X = df[selected_features]
y = df['Label']

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to tensors
X_train_tensor = torch.FloatTensor(X_train).to(device)
y_train_tensor = torch.LongTensor(y_train.values).to(device)
X_test_tensor = torch.FloatTensor(X_test).to(device)
y_test_tensor = torch.LongTensor(y_test.values).to(device)


In [ ]:

# Initialize environment and agent
env = CarHackingEnv(X_train, y_train.values)
state_size = X_train.shape[1]
action_size = 2
agent = DQNAgent(state_size, action_size)

# Training parameters
episodes = 300
update_target_every = 10
batch_size = 128

# Training loop
best_accuracy = 0
for episode in range(episodes):
    state = env.reset()
    total_reward = 0
    done = False

    while not done:
        action = agent.act(state)
        next_state, reward, done, info = env.step(action)
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        total_reward += reward

        if done:
            break

    # Experience replay
    for _ in range(5):  # Multiple updates per episode
        agent.replay()

    agent.decay_epsilon()

    # Update target network periodically
    if episode % update_target_every == 0:
        agent.update_target_model()

    # Evaluation
    if episode % 10 == 0:
        with torch.no_grad():
            test_outputs = agent.model(X_test_tensor)
            _, predicted = torch.max(test_outputs, 1)
            accuracy = accuracy_score(y_test_tensor.cpu(), predicted.cpu())

            if accuracy > best_accuracy:
                best_accuracy = accuracy
                torch.save(agent.model.state_dict(), 'best_dqn_model.pth')

            print(f"Episode {episode}, Reward: {total_reward:.2f}, Test Accuracy: {accuracy:.4f}, Epsilon: {agent.epsilon:.4f}")


Episode 0, Reward: 1205970.00, Test Accuracy: 0.7935, Epsilon: 0.9950
Episode 10, Reward: 1369440.00, Test Accuracy: 0.9541, Epsilon: 0.9464


KeyboardInterrupt: 